In [80]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select


In [93]:
aqi_indexes = [1,2,3,4,5]
years = ['2011','2012']
months = [
'January',
'February',
'March',
'April',
'May',
'June',
'July',
'August',
'September',
'October',
'November',
'December',]


In [101]:
def get_aqi(soup,year,month):
     """
     Assumes:
          soup is a BeautifulSoup instance
          year string represnting year in format YYYY
          month, string representing month name, capitalized
     Returns:
          None
     """ 
     readings = []
     for index in aqi_indexes:
          for element in soup.findAll(attrs={'class':f'aqi{index}'}):
               l_el = list(element)
               readings.append([l_el[0],l_el[2]])
     results.append({'Year':year,'Month':month,'Readings': readings})


def update_data(year, month):
    """
    Assumes:
         year string represnting year in format YYYY
         month, string representing month name, capitalized
    Returns:
         None
    """
    select_year = Select(driver.find_element_by_id('year'))
    select_year.select_by_visible_text(year)
    select_month = Select(driver.find_element_by_id('month'))
    select_month.select_by_visible_text(month)
    driver.find_element_by_id('viewCal').click()
    content = driver.page_source

    get_aqi(BeautifulSoup(content),year,month)

In [103]:
results=[]
driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver')
url ='http://www.cleanairpartners.net/historical-air-quality'
driver.get(url)
frame = driver.find_element_by_id('historicalFrame')
driver.switch_to.frame(frame)
for year in years:
    for month in months:
        update_data(year,month)
driver.quit()

In [116]:
import pandas as pd
import numpy as np
from datetime import datetime
columns = ['dteday','yr','mnth','weekday','aqi_label','aqi_reading']
years = {'2011':0,'2012':1}
months = {
'January':1,
'February':2,
'March':3,
'April':4,
'May':5,
'June':6,
'July':7,
'August':8,
'September':9,
'October':10,
'November':11,
'December':12}

def aqi_labels(aqi):
    if aqi<=50:
        return 1
    elif aqi<=100:
        return 2
    elif aqi<=150:
        return 3
    elif aqi<=200:
        return 4
    else:
        return 5
rows = []

for result in results:
    for entry in result['Readings']:
        year = int(result['Year'])
        month = months[result['Month']]
        day = int(entry[0])
        aqi = int(entry[1].split(':')[1])
        entry_date = datetime(year,month, day)
        rows.append([entry_date,
        entry_date.year,
        entry_date.month,
        entry_date.weekday(),
        aqi_labels(aqi),
        aqi])
df = pd.DataFrame(rows, columns = columns)

In [123]:
df.sort_values('dteday', ascending=True, inplace=True, axis=0)
df.reset_index(drop = True, inplace = True)

In [125]:
df.to_csv('pollution.csv')